In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_3c4e2af75bb0495ab77132958276fffa.csv"
df=pd.read_csv(url)
df.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,Variab10.txt,35400,CTG0001.txt,240,357,120,120,0,0,0,...,0,0,0,0,0,0,1,0,9,2
1,Fmcs_1.txt,35188,CTG0002.txt,5,632,132,132,4,0,4,...,0,0,0,1,0,0,0,0,6,1
2,Fmcs_1.txt,35188,CTG0003.txt,177,779,133,133,2,0,5,...,0,0,0,1,0,0,0,0,6,1
3,Fmcs_1.txt,35188,CTG0004.txt,411,1192,134,134,2,0,6,...,0,0,0,1,0,0,0,0,6,1
4,Fmcs_1.txt,35188,CTG0005.txt,533,1147,132,132,4,0,5,...,0,0,0,0,0,0,0,0,2,1


In [2]:
df=df[["LB","AC","FM","UC","DL","DS","DP","ASTV","MSTV","ALTV","MLTV","Width","Min","Max","Nmax","Nzeros","Mode","Mean","Median","Variance","Tendency","NSP"]]
df.head()

,LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,NSP
0,120,0,0,0,0,0,0,73,0.5,43,...,62,126,2,0,120,137,121,73,1,2
1,132,4,0,4,2,0,0,17,2.1,0,...,68,198,6,1,141,136,140,12,0,1
2,133,2,0,5,2,0,0,16,2.1,0,...,68,198,5,1,141,135,138,13,0,1
3,134,2,0,6,2,0,0,16,2.4,0,...,53,170,11,0,137,134,137,13,1,1
4,132,4,0,5,0,0,0,16,2.4,0,...,53,170,9,0,137,136,138,11,1,1


In [3]:
import numpy as np
y=df[["NSP"]]
y=y.replace(1,0)
y=y.replace([2,3],1)
np.unique(y)

array([0, 1])

In [4]:
y.groupby("NSP")["NSP"].count()

NSP
0    1655
1     471
Name: NSP, dtype: int64

In [5]:
X=df.drop(["NSP"],axis=1)
X.head()

,LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Width,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency
0,120,0,0,0,0,0,0,73,0.5,43,...,64,62,126,2,0,120,137,121,73,1
1,132,4,0,4,2,0,0,17,2.1,0,...,130,68,198,6,1,141,136,140,12,0
2,133,2,0,5,2,0,0,16,2.1,0,...,130,68,198,5,1,141,135,138,13,0
3,134,2,0,6,2,0,0,16,2.4,0,...,117,53,170,11,0,137,134,137,13,1
4,132,4,0,5,0,0,0,16,2.4,0,...,117,53,170,9,0,137,136,138,11,1


In [6]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
7,683.425325
9,668.944697
1,335.780567
6,278.825050
0,143.875993
3,101.551012
8,95.774986
10,65.155619
12,54.501647
11,43.823963


In [7]:
X_score=scores[scores["scores"]>=50]
len(X_score)

9

In [131]:
X_prime=X.iloc[:,X_score.index]
X_prime.head()

,ASTV,ALTV,AC,DP,LB,UC,MSTV,MLTV,Min
0,73,43,0,0,120,0,0.5,2.4,62
1,17,0,4,0,132,4,2.1,10.4,68
2,16,0,2,0,133,5,2.1,13.4,68
3,16,0,2,0,134,6,2.4,23.0,53
4,16,0,4,0,132,5,2.4,19.9,53


In [132]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_prime=pd.DataFrame(scaler.fit_transform(X_prime),columns=X_prime.columns)
X_prime.head()

,ASTV,ALTV,AC,DP,LB,UC,MSTV,MLTV,Min
0,1.513190,1.802542,-0.764740,-0.27153,-1.352220,-1.285798,-0.943095,-1.028560,-1.068562
1,-1.744751,-0.535361,0.358852,-0.27153,-0.132526,0.119475,0.868841,0.393176,-0.865539
2,-1.802928,-0.535361,-0.202944,-0.27153,-0.030884,0.470793,0.868841,0.926327,-0.865539
3,-1.802928,-0.535361,-0.202944,-0.27153,0.070757,0.822111,1.208579,2.632411,-1.373097
4,-1.802928,-0.535361,0.358852,-0.27153,-0.132526,0.470793,1.208579,2.081488,-1.373097


In [133]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1594, 9)
(532, 9)


In [134]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
BBagging=BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=250),n_estimators=15,
                                   sampling_strategy="not majority",
                                   random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BBagging

BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=250),
                          n_estimators=15, random_state=0,
                          sampling_strategy='not majority')

In [135]:
from sklearn.metrics import accuracy_score
train_pred=BBagging.predict(X_train)
test_pred=BBagging.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9937264742785445
Accuracy on test data:  0.9548872180451128


In [136]:
from sklearn.metrics import roc_auc_score
print("ROC_AUC on train data: ",roc_auc_score(y_train,train_pred))
print("ROC_AUC on test data: ",roc_auc_score(y_test,test_pred))

ROC_AUC on train data:  0.9878312106572976
ROC_AUC on test data:  0.9111237715425154


In [137]:
from sklearn.metrics import precision_score
print("Precision score on train data: ",precision_score(y_train,train_pred,average="binary"))
print("Precision score on test data: ",precision_score(y_test,test_pred,average="binary"))

Precision score on train data:  0.9942196531791907
Precision score on test data:  0.9611650485436893


In [138]:
from sklearn.metrics import recall_score
print("Recall score on train data: ",recall_score(y_train,train_pred,average="binary"))
print("Recall score on test data: ",recall_score(y_test,test_pred,average="binary"))

Recall score on train data:  0.9772727272727273
Recall score on test data:  0.8319327731092437


In [139]:
from sklearn.metrics import f1_score
print("F1 score on train data: ",f1_score(y_train,train_pred,average="binary"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="binary"))

F1 score on train data:  0.98567335243553
F1 score on test data:  0.8918918918918919


In [140]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for train data: \n",confusion_matrix(y_train,train_pred))
print("Confusion matrix for test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix for train data: 
 [[1240    2]
 [   8  344]]
Confusion matrix for test data: 
 [[409   4]
 [ 20  99]]


In [141]:
print("Specificity for test data: ",329/(329+4))
print("Sensitivity for test data: ",78/(78+15))

Specificity for test data:  0.987987987987988
Sensitivity for test data:  0.8387096774193549


In [142]:
from sklearn.ensemble import GradientBoostingClassifier
BBagging=BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(loss="exponential",n_estimators=250),
                                   n_estimators=15,
                                   sampling_strategy="not majority",
                                   random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BBagging

BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(loss='exponential',
                                                                    n_estimators=250),
                          n_estimators=15, random_state=0,
                          sampling_strategy='not majority')

In [143]:
from sklearn.metrics import accuracy_score
train_pred=BBagging.predict(X_train)
test_pred=BBagging.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9868255959849436
Accuracy on test data:  0.9454887218045113


In [144]:
from sklearn import svm
BBagging=BalancedBaggingClassifier(base_estimator=svm.SVC(C=1.5,kernel="poly",decision_function_shape="ovo"),
                                   n_estimators=15,
                                   sampling_strategy="not majority",
                                   random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BBagging

BalancedBaggingClassifier(base_estimator=SVC(C=1.5,
                                             decision_function_shape='ovo',
                                             kernel='poly'),
                          n_estimators=15, random_state=0,
                          sampling_strategy='not majority')

In [145]:
train_pred=BBagging.predict(X_train)
test_pred=BBagging.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9322459222082811
Accuracy on test data:  0.9285714285714286


In [180]:
from sklearn.neighbors import KNeighborsClassifier
BBagging=BalancedBaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=2,weights="distance",algorithm="ball_tree",
                                                                      p=1),
                                   n_estimators=15,sampling_strategy="not majority",
                                  random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BBagging

BalancedBaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='ball_tree',
                                                              n_neighbors=2,
                                                              p=1,
                                                              weights='distance'),
                          n_estimators=15, random_state=0,
                          sampling_strategy='not majority')

In [181]:
train_pred=BBagging.predict(X_train)
test_pred=BBagging.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9974905897114178
Accuracy on test data:  0.9511278195488722


** *********************** **

In [16]:
X_score=scores[scores["scores"]>=40]
len(X_score)

10

In [17]:
X_prime=X.iloc[:,X_score.index]
X_prime.head()

,ASTV,ALTV,AC,DP,LB,UC,MSTV,MLTV,Min,Width
0,73,43,0,0,120,0,0.5,2.4,62,64
1,17,0,4,0,132,4,2.1,10.4,68,130
2,16,0,2,0,133,5,2.1,13.4,68,130
3,16,0,2,0,134,6,2.4,23.0,53,117
4,16,0,4,0,132,5,2.4,19.9,53,117


In [18]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_prime=pd.DataFrame(scaler.fit_transform(X_prime),columns=X_prime.columns)
X_prime.head()

,ASTV,ALTV,AC,DP,LB,UC,MSTV,MLTV,Min,Width
0,1.513190,1.802542,-0.764740,-0.27153,-1.352220,-1.285798,-0.943095,-1.028560,-1.068562,-0.165507
1,-1.744751,-0.535361,0.358852,-0.27153,-0.132526,0.119475,0.868841,0.393176,-0.865539,1.529124
2,-1.802928,-0.535361,-0.202944,-0.27153,-0.030884,0.470793,0.868841,0.926327,-0.865539,1.529124
3,-1.802928,-0.535361,-0.202944,-0.27153,0.070757,0.822111,1.208579,2.632411,-1.373097,1.195333
4,-1.802928,-0.535361,0.358852,-0.27153,-0.132526,0.470793,1.208579,2.081488,-1.373097,1.195333


In [138]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1594, 10)
(532, 10)


In [146]:
y_test.groupby("NSP")["NSP"].count()

NSP
0    413
1    119
Name: NSP, dtype: int64

In [148]:
print("Minority proportion in test data: ",119/(119+413))

Minority proportion in test data:  0.2236842105263158


In [199]:
from imblearn.ensemble import BalancedRandomForestClassifier
BForest=BalancedRandomForestClassifier(class_weight="balanced_subsample",sampling_strategy="not majority",n_jobs=-1,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(class_weight='balanced_subsample', n_jobs=-1,
                               random_state=0,
                               sampling_strategy='not majority')

In [200]:
from sklearn.metrics import accuracy_score
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  1.0
Accuracy on test data:  0.9661654135338346


In [206]:
from sklearn.metrics import roc_auc_score
print("ROC_AUC on train data: ",roc_auc_score(y_train,train_pred))
print("ROC_AUC on test data: ",roc_auc_score(y_test,test_pred))

ROC_AUC on train data:  1.0
ROC_AUC on test data:  0.9333428286568866


In [202]:
from sklearn.metrics import f1_score
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  1.0
F1 score on test data:  0.9661654135338346


In [203]:
from sklearn.metrics import precision_score
print("Precision score on train data: ",precision_score(y_train,train_pred,average="binary"))
print("Precision score on test data: ",precision_score(y_test,test_pred,average="binary"))

Precision score on train data:  1.0
Precision score on test data:  0.9719626168224299


In [204]:
from sklearn.metrics import recall_score
print("Recall score on train data: ",recall_score(y_train,train_pred,average="micro"))
print("Recall score on test data: ",recall_score(y_test,test_pred,average="micro"))

Recall score on train data:  1.0
Recall score on test data:  0.9661654135338346


In [101]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for train data: \n",confusion_matrix(y_train,train_pred))
print("Confusion matrix for test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix for train data: 
 [[1242    0]
 [   0  352]]
Confusion matrix for test data: 
 [[410   3]
 [ 15 104]]


In [102]:
print("Specificity for test data: ",410/(410+3))
print("Sensitivity for test data: ",104/(104+15))

Specificity for test data:  0.9927360774818402
Sensitivity for test data:  0.8739495798319328


In [124]:
y_pred=pd.DataFrame(BForest.predict_proba(X_test),columns=["pred_0","pred_1"])
y_pred.head()

,pred_0,pred_1
0,1.00,0.00
1,0.99,0.01
2,0.85,0.15
3,1.00,0.00
4,0.94,0.06


In [125]:
y_probs1=y_pred[["pred_1"]]

In [126]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [127]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [132]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs1,t),average="micro") for t in thresholds]

In [133]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.491, 0.9661654135338346)


In [134]:
len(y_probs1[y_probs1["pred_1"]>=0.491])

107

No more room for improvements.

*** ********************** ***

In [108]:
from imblearn.ensemble import BalancedRandomForestClassifier
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [127]:
BForest=BalancedRandomForestClassifier(n_estimators=100,criterion="entropy",sampling_strategy="not majority",random_state=0)
scores=cross_val_score(BForest,X_prime,np.asarray(y).reshape(y.shape[0],),cv=6)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8842484284236493 +/- 0.08744635693663884


** ***************  **

In [182]:
X_score=scores[scores["scores"]>=10]
len(X_score)

13

In [183]:
X_prime=X.iloc[:,X_score.index]
X_prime.head()

,ASTV,ALTV,AC,DP,LB,UC,MSTV,MLTV,Min,Width,Mode,DS,Variance
0,73,43,0,0,120,0,0.5,2.4,62,64,120,0,73
1,17,0,4,0,132,4,2.1,10.4,68,130,141,0,12
2,16,0,2,0,133,5,2.1,13.4,68,130,141,0,13
3,16,0,2,0,134,6,2.4,23.0,53,117,137,0,13
4,16,0,4,0,132,5,2.4,19.9,53,117,137,0,11


In [184]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_prime=pd.DataFrame(scaler.fit_transform(X_prime),columns=X_prime.columns)
X_prime.head()

,ASTV,ALTV,AC,DP,LB,UC,MSTV,MLTV,Min,Width,Mode,DS,Variance
0,1.513190,1.802542,-0.764740,-0.27153,-1.352220,-1.285798,-0.943095,-1.028560,-1.068562,-0.165507,-1.065614,-0.057476,1.870569
1,-1.744751,-0.535361,0.358852,-0.27153,-0.132526,0.119475,0.868841,0.393176,-0.865539,1.529124,0.216638,-0.057476,-0.234998
2,-1.802928,-0.535361,-0.202944,-0.27153,-0.030884,0.470793,0.868841,0.926327,-0.865539,1.529124,0.216638,-0.057476,-0.200481
3,-1.802928,-0.535361,-0.202944,-0.27153,0.070757,0.822111,1.208579,2.632411,-1.373097,1.195333,-0.027600,-0.057476,-0.200481
4,-1.802928,-0.535361,0.358852,-0.27153,-0.132526,0.470793,1.208579,2.081488,-1.373097,1.195333,-0.027600,-0.057476,-0.269516


In [212]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1594, 13)
(532, 13)


In [186]:
#train=pd.concat([X_train,y_train],axis=1)
#train0=train[train["NSP"]==0]
#train1=train[train["NSP"]==1]
#train1=pd.concat([train1]*2,axis=0)
#train=pd.concat([train0,train1],axis=0)
#X_train=train.drop(["NSP"],axis=1)
#y_train=train[["NSP"]]
#print(X_train.shape)

In [213]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(bootstrap=False,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(bootstrap=False, random_state=0)

In [214]:
from sklearn.metrics import accuracy_score
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  1.0
Accuracy on test data:  0.9605263157894737


In [215]:
from sklearn.metrics import roc_auc_score
print("ROC_AUC on train data: ",roc_auc_score(y_train,train_pred))
print("ROC_AUC on test data: ",roc_auc_score(y_test,test_pred))

ROC_AUC on train data:  1.0
ROC_AUC on test data:  0.9267198404785644


In [216]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for train data: \n",confusion_matrix(y_train,train_pred))
print("Confusion matrix for test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix for train data: 
 [[1242    0]
 [   0  352]]
Confusion matrix for test data: 
 [[408   5]
 [ 16 103]]


In [217]:
print("Test data specificity: ",408/(408+5))
print("Test data sensitivity: ",103/(103+16))

Test data specificity:  0.9878934624697336
Test data sensitivity:  0.865546218487395


In [218]:
y_pred=pd.DataFrame(Forest.predict_proba(X_test),columns=["pred_0","pred_1"])
y_pred.head()

,pred_0,pred_1
0,1.00,0.00
1,1.00,0.00
2,0.96,0.04
3,1.00,0.00
4,1.00,0.00


In [219]:
y_probs1=y_pred[["pred_1"]]

In [220]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [221]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [222]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs1,t),average="micro") for t in thresholds]

In [223]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.491, 0.9624060150375939)


In [224]:
len(y_probs1[y_probs1["pred_1"]>=0.491])

109

In [225]:
y_pred["pred_0"]=np.where((y_pred.pred_0 >=0.491),1,y_pred.pred_0)
y_pred["pred_0"]=np.where((y_pred.pred_0 < 0.491),0,y_pred.pred_0)
y_pred["pred_1"]=np.where((y_pred.pred_1 >=0.491),1,y_pred.pred_1)
y_pred["pred_1"]=np.where((y_pred.pred_1 <0.491),0,y_pred.pred_1)

In [226]:
len11=len(y_pred[y_pred["pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  109


In [227]:
y_pred["pred_0"]=np.where((y_pred.pred_0 == y_pred.pred_1),0,y_pred.pred_0)

In [228]:
len00=len(y_pred[y_pred["pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  423


In [229]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(handle_unknown="ignore")
y_true=pd.DataFrame(enc.fit_transform(y_test).toarray())

In [230]:
y_pred=enc.inverse_transform(y_pred)

In [231]:
print("AUC after adjusting for the threshold: ",roc_auc_score(y_test,y_pred))

AUC after adjusting for the threshold:  0.9309215211508332


In [232]:
print("Confusion_matrix after adjusting the threshold: \n",confusion_matrix(y_test,y_pred))

Confusion_matrix after adjusting the threshold: 
 [[408   5]
 [ 15 104]]


In [205]:
print("Test data accuracy after adjusting the threshold is: ",(408+104)/(408+104+5+15))

Test data accuracy after adjusting the threshold is:  0.9624060150375939


In [233]:
print("Specificity after adjusting the threshold: ",408/(408+5))
print("Sensitivity after adjusting the threshold: ",104/(104+15))

Specificity after adjusting the threshold:  0.9878934624697336
Sensitivity after adjusting the threshold:  0.8739495798319328


### The best model is BalancedRandomForestClassifier. The second best one is RandomForestClassifier. 